In [1]:
import re
import math
import nltk
import pandas as pd
import numpy as np
from reynir import Greynir
from googletrans import Translator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import NaiveBayesClassifier
from nltk import stem
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

t = Translator()
g = Greynir()

def google_translate(text):
    text = text.replace("<br />", "")
    return t.translate(text, dest="is").text

data = pd.read_csv('IMDB-Dataset.csv')
icelandic = google_translate(data['review'][0])

print(data['review'][0].replace('<br />', ''))



One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wou

In [4]:


print(icelandic[0:100])

    

Einn hinna gagnrýnenda hefur nefnt að eftir að hafa horft aðeins á 1 Oz þátt þá verðurðu hrifinn. Þe


In [136]:
txt = "Einn hinna gagnrýnenda hefur nefnt að eftir að hafa horft aðeins á 1 Oz þátt þá verðurðu hrifinn."
parse = g.parse_tokens(txt)

print(dir(parse))
print(parse.parse())
print(parse.lemmas)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_err_index', '_error', '_job', '_len', '_num', '_s', '_score', '_simplified_tree', '_terminals', '_tree', 'categories', 'combinations', 'deep_tree', 'dump', 'dumps', 'err_index', 'error', 'flat_tree', 'ifd_tags', 'is_foreign', 'lemmas', 'lemmas_and_cats', 'lemmas_mm', 'load', 'loads', 'parse', 'score', 'terminal_nodes', 'terminals', 'text', 'tidy_text', 'tokens', 'tree']
False
None


In [91]:
#data_ice_google = pd.read_csv("IMDB-Dataset-GoogleTranslate.csv")







,review,sentiment
0,Hvattur af jákvæðum ummælum um þessa mynd hérn...,negative
1,Einn hinna gagnrýnenda hefur nefnt að eftir að...,positive
2,Yndisleg lítil framleiðsla. Kvikmyndatæknin er...,positive
3,"""Love in the Time of Money"" eftir Petter Matte...",positive
4,Í grundvallaratriðum er fjölskylda þar sem lít...,negative


In [27]:

dataset = pd.read_csv("IMDB-Dataset-GoogleTranslate.csv")
reviews = dataset['review'].values
labels = dataset['sentiment'].values

def get_sentiment(s):
    return {'sentiment': s}

featuresets = [(get_sentiment(sentiment), review) for review, sentiment in zip(reviews, labels)]
train_set, test_set = featuresets[35000:], featuresets[35000:50000]
classifier = NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
#train_set
#get_sentiment("positive")
#encoder = LabelEncoder()
#encoded_labels = encoder.fit_transform(labels)
#train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify=encoded_labels)


In [32]:
vec = CountVectorizer(max_features=3000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names_out()
X = X.toarray()
word_counts = {}
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]


In [33]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data
    
def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors



In [36]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab:
                continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result
    

In [37]:
labels = [0, 1]
n_label_items, log_label_priors = fit(train_sentences, train_labels, labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy: ", accuracy_score(test_labels, pred))

Accuracy:  0.7892100509114797


In [65]:
def predict2(text):
    if predict(n_label_items, vocab, word_counts, log_label_priors, labels, [text]) == [0]:
        return text, ' == [Neikvætt]'
    elif predict(n_label_items, vocab, word_counts, log_label_priors, labels, [text]) == [1]:
        return text, ' == [Jákvætt]'



('Þessi mynd er geðveikt leiðinleg', ' == [Neikvætt]')
('Ég horfði á þessa mynd og verð að segja að mér fannst hún ekki skemmtileg', ' == [Jákvætt]')
('Mér fannst þessi mynd satt að segja vera mjög skemmtileg', ' == [Jákvætt]')


In [10]:


#classifier = NaiveBayesClassifier.train(



array(['Maður að nafni Jerry kemst í eigu fornrar Azteka dúkku. Hins vegar er þessi hrollvekjandi litla persóna haldin illum anda, sem tekur yfir líkama Jerrys og ýtir honum til að hella út blóðinu...Ég hef séð tvær aðrar hryllingsmyndir „The Dorm that Dipped Blood" og "The Kindred" gerð af Stephen Carpenter og Jeffrey Obrow og ég verð að segja að "The Power" veldur ekki vonbrigðum heldur. Söguþráðurinn er hægfara, en það eru nokkur áhrifarík mannleg goo áhrif og svolítið af gore.Senan þar sem kvenkyns blaðamaður verður fyrir árás af vopnum sem rífa úr dýnu er kjaftæði. Mér líkaði við þessa lággjalda hryllingsmynd og þú ættir líka að gera það ef þú ert í 80\'s hryllingstegundinni.8 af 10 .',
       'Það eru töluverðir peningar á bak við þessa framleiðslu, þannig að útlitið á henni er mjög gott. Það inniheldur nokkrar áhugaverðar framkomur eftir Gilbert Roland, Eddie Burns, og stutt mynd í upphafi eftir Christopher Lee. Það eru nokkrir spennandi skotbardagar, og gamansamir hlutir eða tv